# My hangman file

In [3]:
import json
import numpy as np
import math
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist

In [2]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [3]:
random.choice(full_dict)

'magyarization'

In [4]:
full_dict_common_letter_sorted = collections.Counter("".join(full_dict)).most_common()

In [5]:
full_dict_common_letter_sorted[0]

('e', 233745)

In [15]:
np.log(233745)

12.361986056547824

Ok, the main things I am revising are the n-gram matching and the evaluate guesses. What do they have to do?
-Ok, I match the n-grams in a different way than other people do it, but I think my way is better.
-I need to pad out the left and right for the n-gram for the generated word. The padding is to match for n-grams at the beginning and end I guess
    -maybe I shouldnt do this...
    -However, I have to discount stuff that is like (None a) or (a None) because those will cause issues with the 
-matching only can reccommend ONE letter. the way it is done in the other programs is forcing it into cases... but I can do it a different way.

I am going to break the functions down here, fix them, and paste them back...


In [6]:
full_dictionary = full_dict

#set up n-grams for current dictionary
grams2 = []
grams3 = []
grams4 = []
grams5 = []
for word in full_dictionary:
    #is the word clean? It should be...
    #when I loaded the data it's fine...
    #the API has be a bit nervous so lets just put this here
    word = word.replace(" ", "") #remove any padding that may mess with this

    grams2.extend(list(ngrams(word, 2))) #if I pad left and match NONES with the length 2 n-grams, the program will start guessing first and last letters first...
    grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
    grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
    grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))

#Ok, so now need the frequency distributions for these
freq_dist_2 = FreqDist(grams2)
freq_dist_3 = FreqDist(grams3)
freq_dist_4 = FreqDist(grams4)
freq_dist_5 = FreqDist(grams5)

#need to actually extract the data from these distributions to use it
freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

words_by_length = {}
min_word_length = 3 #shortest in the full_dict at least
max_word_length = 40 #I guess this is pretty arbitrary

for i in range(min_word_length, max_word_length):
    words_by_length.update({i : []})

#I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
#for word in self.full_dictionary:
#    self.full_dictionary[len(word)].append(word)
    

#manually input a word and guess word...
correct_word = "apple"
word = "_pp_e"
guess_list = ["s", "p", "t", "e"]
#failed_letters = ["s", "t"]


#def update_grams(self, word, guess_list):
        #update ngrams to remove items associated with incorrect letters
temp_word = word.replace("_", "")
incorrect_letters = set(guess_list) - set(temp_word)
freq_dist_2 = [(item[0], item[1]) for item in freq_dist_2 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_3 = [(item[0], item[1]) for item in freq_dist_3 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_4 = [(item[0], item[1]) for item in freq_dist_4 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_5 = [(item[0], item[1]) for item in freq_dist_5 if set(item[0]).isdisjoint(set(incorrect_letters))]



    

In [7]:
def find_potential_guesses(freq_N, input_word, guess_list):
        #This takes as input a freq dist for n-grams and a word
        #it outputs all the n-grams from freq dist that match along with the 
        potential_guesses = []
        filtered_dist = []
        if(freq_N == []):
            dimension = 0
        else:
            dimension = len(freq_N[0][0])
        word_ngrams = list(ngrams(input_word, dimension, pad_left = True, pad_right = True))
        #print(dimension)
        #There is an issue though, too much padding, so I will prune the word ngrams to prevent weird overmatching of NONEs
        #print(item[0] for item in word_ngrams)
        #print(word_ngrams)
        word_ngrams = [item for item in word_ngrams if item.count(None) < 2]

        for fe in freq_N:
            #Ok..... so here is the idea...
            #if the ngram in fe matches, then it must be a match 
            #to one of the n-gons of the target word with wildcards
            fe_gram = fe[0]
            occurrences = fe[1]
            
            #now, try to match it to one of the n-grams of the word...
            matched_letters_count = 0 #you must match at least one letter to be considered

            #how about this, I will potentially return guesses that are already in the guess_list
            #and then filter them out in the next program when I go to evaluate the guesses.
            #maybe also worry about the vowel thing there instead of here.

            remove_set = set(guess_list)
            remove_set.add(None)
            
            #print("I am inside the if statement")
            for word_gram in word_ngrams:
                matched_letters_count = 0
                for i in range(dimension):
                    if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                        if(fe_gram[i] == word_gram[i]):
                            matched_letters_count += 1
                        else:
                            #this happens if we have a direct contradiction, immediately reject this
                            matched_letters_count = 0
                            break
                    #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                if(matched_letters_count == dimension - 1): #there should be n-1 matches and 1 guess
                    potential_guesses.append((set(fe_gram) - remove_set, dimension,occurrences))
                    #this is just for testing below vvvvvvvvv
                    #potential_guesses.append((set(fe_gram) - remove_set, fe_gram, dimension,occurrences))

                    #filtered_dist.append(fe)
                    #print(set(fe_gram).issubset(guess_list))
            
        return potential_guesses

In [8]:
find_potential_guesses(freq_dist_2, word, guess_list)

[({'r'}, 2, 24800),
 ({'l'}, 2, 22135),
 ({'n'}, 2, 19584),
 ({'d'}, 2, 13954),
 ({'h'}, 2, 11707),
 ({'m'}, 2, 11077),
 ({'v'}, 2, 10593),
 (set(), 2, 10209),
 (set(), 2, 10209),
 ({'c'}, 2, 9023),
 ({'h'}, 2, 8865),
 ({'r'}, 2, 8590),
 ({'i'}, 2, 8301),
 ({'o'}, 2, 8175),
 ({'g'}, 2, 7326),
 ({'a'}, 2, 7313),
 ({'o'}, 2, 7255),
 ({'i'}, 2, 6063),
 ({'a'}, 2, 6038),
 ({'b'}, 2, 5974),
 (set(), 2, 5426),
 ({'k'}, 2, 5188),
 (set(), 2, 5034),
 ({'l'}, 2, 4323),
 ({'i'}, 2, 4121),
 ({'m'}, 2, 3951),
 ({'z'}, 2, 3531),
 ({'a'}, 2, 3350),
 ({'f'}, 2, 3328),
 ({'w'}, 2, 3228),
 ({'u'}, 2, 2942),
 (set(), 2, 2260),
 (set(), 2, 2260),
 ({'u'}, 2, 2259),
 ({'u'}, 2, 2223),
 ({'y'}, 2, 2029),
 ({'r'}, 2, 2021),
 ({'o'}, 2, 1715),
 ({'n'}, 2, 1403),
 ({'y'}, 2, 1380),
 ({'y'}, 2, 970),
 ({'l'}, 2, 950),
 ({'j'}, 2, 735),
 ({'x'}, 2, 638),
 ({'x'}, 2, 540),
 ({'n'}, 2, 304),
 ({'d'}, 2, 190),
 ({'b'}, 2, 145),
 ({'m'}, 2, 141),
 ({'h'}, 2, 140),
 ({'w'}, 2, 124),
 ({'f'}, 2, 123),
 ({'b'}, 2, 122

There are issues with the padding... I need to prune everything that has at least 2 NONEs. That includes the `freq_n` and the ngrams of the word...
Maybe I just need to prune the ones from the word?

In [9]:
find_potential_guesses(freq_dist_4, word, guess_list)

[({'a'}, 4, 275),
 ({'l'}, 4, 135),
 ({'l'}, 4, 122),
 ({'r'}, 4, 119),
 ({'i'}, 4, 66),
 ({'o'}, 4, 56),
 ({'i'}, 4, 42),
 ({'u'}, 4, 32),
 ({'h'}, 4, 26),
 (set(), 4, 26),
 (set(), 4, 25),
 (set(), 4, 10),
 (set(), 4, 5),
 ({'n'}, 4, 4),
 ({'a'}, 4, 4),
 ({'r'}, 4, 4),
 (set(), 4, 4),
 ({'o'}, 4, 3),
 ({'u'}, 4, 3),
 ({'a'}, 4, 2),
 ({'n'}, 4, 2),
 ({'c'}, 4, 2),
 ({'d'}, 4, 1),
 ({'l'}, 4, 1),
 ({'m'}, 4, 1),
 ({'n'}, 4, 1),
 ({'d'}, 4, 1),
 (set(), 4, 1),
 ({'y'}, 4, 1),
 ({'h'}, 4, 1),
 ({'j'}, 4, 1),
 ({'k'}, 4, 1)]

In [10]:
find_potential_guesses(freq_dist_5, word, guess_list)

[({'l'}, 5, 37), ({'i'}, 5, 13), (set(), 5, 2), ({'a'}, 5, 1)]

In [11]:
freq_2_guesses = find_potential_guesses(freq_dist_2, word, guess_list)
freq_3_guesses = find_potential_guesses(freq_dist_3, word, guess_list)
freq_4_guesses = find_potential_guesses(freq_dist_4, word, guess_list)
freq_5_guesses = find_potential_guesses(freq_dist_5, word, guess_list)
all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses

In [12]:
print(all_guesses[0:10])

[({'r'}, 2, 24800), ({'l'}, 2, 22135), ({'n'}, 2, 19584), ({'d'}, 2, 13954), ({'h'}, 2, 11707), ({'m'}, 2, 11077), ({'v'}, 2, 10593), (set(), 2, 10209), (set(), 2, 10209), ({'c'}, 2, 9023)]


In [13]:
filtered_guesses = [item for item in all_guesses if item[0] != set()] #this filters out all the guesses with no letters in them
sorted_guesses = sorted(filtered_guesses, key = lambda tup: tup[1], reverse = True)
print(sorted_guesses[0:10])

[({'l'}, 5, 37), ({'i'}, 5, 13), ({'a'}, 5, 1), ({'a'}, 4, 275), ({'l'}, 4, 135), ({'l'}, 4, 122), ({'r'}, 4, 119), ({'i'}, 4, 66), ({'o'}, 4, 56), ({'i'}, 4, 42)]


In [14]:
def evaluate_potential_guesses(potential_guesses):
        #each potential guess is in the form:
        #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

        #So, which do we pick:
        #prioritize ngram length first
        #then pick based on most frequent letter
        #I guess how many letters match isnt as important...
        #I have read that vowel ratio may make a difference... maybe implement later

        sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
        if(sorted_guesses != []):
            max_dim = sorted_guesses[0][1]
            #print(max_dim)
            max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
            max_dim_guesses.sort(key = lambda tup: tup[2], reverse = True)

            #print(max_dim_guesses[0])
            max_dim_guesses[0][0]
            max_guess = max_dim_guesses[0]
            #print(max_guess)
            max_guess = list(max_guess)[0] #this is to get the actual string instead of a set
            if len(max_guess) > 0:
                max_guess = list(max_guess)[0]
            
            
            return max_guess #this is a set in general
        else:
            return "!"

In [15]:
evaluate_potential_guesses(all_guesses)

'l'

Ok... now just need a program to sort through all of the different n-grams and pick the best.

In [16]:
from nltk import ngrams, FreqDist

In [23]:
test = "_pp_"
test.count("_")

2

In [28]:
#word = "_pp_e"
#word2 = "app__"
#word = "agrumen_a_ive"
word = "________"

#first pad beginning and end of word with something, will replace with None later
substr_lists = []
word = "." + word + "." # I think this is ok
#word = temp_word
for i,char in enumerate(word):
    if char == "_": #found a blank
        start, end = i, i
        #get longest substring containing the blank
        while start > 0 and word[start-1] != "_":
            start-=1
        while end < len(word) - 1 and word[end+1] != "_":
            end += 1
        #now, turn substring into something to match an n-gram
        substring = word[start:end + 1]
        ssList = list(substring)
        if ssList[0] == ".": ssList[0] = None
        if ssList[-1] == ".": ssList[-1] = None
        #print ssList)
        #what if substr is longer length than 5?
        #NOTE, IF I USE NGRAMS LONGER THAN 5 NEED TO UPDATE THIS!
        if(len(ssList) > 5):
            grams = ngrams(ssList, 5)
            for g in grams:
                #print(g)
                if(g.count("_") == 1):
                    substr_lists.append(g)
            #print(grams)
        elif(set(ssList) - set({None, "_"}) != set()):
            substr_lists.append(ssList)
        
print(substr_lists)


[]


In [7]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []
        self.has_won = False
        self.init_grams()
        #self.init_grams()

    def init_grams(self):
        #set up n-grams for current dictionary
        grams2 = []
        grams3 = []
        grams4 = []
        grams5 = []
        for word in self.full_dictionary:
            #is the word clean? It should be...
            #when I loaded the data it's fine...
            #the API has be a bit nervous so lets just put this here
            word = word.replace(" ", "") #remove any padding that may mess with this

            grams2.extend(list(ngrams(word, 2)))# padding on 2 messes up the guesses.
            grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
            grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
            grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))
        
        #Ok, so now need the frequency distributions for these
        freq_dist_2 = FreqDist(grams2)
        freq_dist_3 = FreqDist(grams3)
        freq_dist_4 = FreqDist(grams4)
        freq_dist_5 = FreqDist(grams5)

        #need to actually extract the data from these distributions to use it
        self.freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
        self.freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
        self.freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
        self.freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

        self.words_by_length = {}
        min_word_length = 3 #shortest in the full_dict at least
        max_word_length = 40 #I guess this is pretty arbitrary

        for i in range(min_word_length, max_word_length):
            self.words_by_length.update({i : []})

        #I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
        #for word in self.full_dictionary:
        #    self.full_dictionary[len(word)].append(word)
        
            
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)



    def find_potential_guesses(self, freq_N_array, input_word, guess_list):
        #This takes as input an array containing each freq_N
        #it outputs all the n-grams from freq dist that match and stuff that will be used to calc the weight
        potential_guesses = []
        filtered_dist = []
        #if(freq_N == []):
        #    return potential_guesses
        #else:
        #    dimension = len(freq_N[0][0])

        remove_set = set(guess_list)
        remove_set.add(None)

        #Ok, so we need to get the longest substrings in the word that contain ONE blank space.
        #first pad beginning and end of word with something, will replace with None later
        substr_lists = []
        input_word = "." + input_word + "." # I think this is ok
        #word = temp_word
        #print(input_word)
        for i,char in enumerate(input_word):
            if char == "_": #found a blank
                start, end = i, i
                #print(i)
                #get longest substring containing the blank
                while start > 0 and input_word[start-1] != "_":
                    start-=1
                    #print(start)
                while end < len(input_word) - 1 and input_word[end+1] != "_":
                    end += 1
                    #print(end)
                #now, turn substring into something to match an n-gram
                substring = input_word[start:end + 1]
                ssList = list(substring)
                if ssList[0] == ".": ssList[0] = None
                if ssList[-1] == ".": ssList[-1] = None
                #print ssList)
                #what if substr is longer length than 5?
                #NOTE, IF I USE NGRAMS LONGER THAN 5 NEED TO UPDATE THIS!
                if(len(ssList) > 5):
                    grams = ngrams(ssList, 5)
                    for g in grams:
                        #print(g)
                        if(g.count("_") == 1):
                            substr_lists.append(g)
                    #print(grams)
                elif(set(ssList) - set({None, "_"}) != set()): #There must be something in the set other than underscores and nones
                    substr_lists.append(ssList)
        for substr in substr_lists: 
            dimension = len(substr)
            freq_N = freq_N_array[dimension-2]
            for fe in freq_N:
                fe_gram = fe[0]
                occurrences = fe[1]

                matched_letters_count = 0
                for i in range(dimension):
                    if(substr[i] != "_"): #there is a letter here, does the fe match?
                            if(fe_gram[i] == substr[i]):
                                matched_letters_count += 1
                            else:
                                matched_letters_count -=1
                                break
                        #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                    if(matched_letters_count == dimension - 1 and len((set(fe_gram) - remove_set)) == 1): #there should be n-1 matches and 1 guess
                        potential_guesses.append((set(fe_gram) - remove_set, dimension, occurrences))
            
        return potential_guesses
        

    def evaluate_potential_guesses(self, potential_guesses):
        print(potential_guesses)
        if(potential_guesses == []):
            return "!" # no n-gram fits
        # Initialize dictionary to keep scores for each guess
        scores = {}

        # Weighting factors, these are the main thing to tweak...
        weight_frequency = 0.25
        weight_length = 2.0  # Giving more weight to the length of the n-gram

        for guess, length, occurrences in potential_guesses:
            guess_letter = list(guess)[0]  # Extract the letter from the set

            # Calculate the score for this guess
            score = (np.log(occurrences) * weight_frequency) + (length * weight_length)

            # Add or update the score for this letter in the scores dictionary
            if guess_letter in scores:
                scores[guess_letter] += score
            else:
                scores[guess_letter] = score

        # Determine the best guess by finding the maximum score
        #print(scores)
        best_guess = max(scores, key=scores.get)
        return best_guess



    
    def update_grams(self, freq_dist_list ,word, guess_list):
        #freq_grams is in the form [freq_dist_2, freq_dist_3, freq_dist_4, freq_dist_5]

        #update ngrams to remove items associated with incorrect letters
        word = word.replace("_", "")
        incorrect_letters = set(guess_list) - set(word)
        for freq_dist in freq_dist_list:
            freq_dist = [(item[0], item[1]) for item in freq_dist if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_2 = [(item[0], item[1]) for item in freq_dist_2 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_3 = [(item[0], item[1]) for item in freq_dist_3 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_4 = [(item[0], item[1]) for item in freq_dist_4 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_5 = [(item[0], item[1]) for item in freq_dist_5 if set(item[0]).isdisjoint(set(incorrect_letters))]
        return freq_dist_list



    def guess(self, word, freq_dist_list): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        reMatchWord = clean_word.replace("_",".")
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(reMatchWord, dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        #print(sorted_letter_count)                
        
        guess_letter = '!'

        all_guesses = self.find_potential_guesses(freq_dist_list, word, self.guessed_letters)
        ngram_guess = self.evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        #print(sorted_letter_count)
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                most_common_guess = letter
                #print(most_common_guess)
                break
        
        
        if(ngram_guess != "!"):
            return ngram_guess
        else:
            return most_common_guess


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word

    def get_winstate(self):
        return self.has_won
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        #self.init_grams()
        freq_dist_list = [self.freq_dist_2, self.freq_dist_3, self.freq_dist_4, self.freq_dist_5]
        self.has_won = False
        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word, freq_dist_list)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)

            freq_dist_list = self.update_grams(freq_dist_list, word, self.guess_list)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                



In [9]:
game = MyHangman()

In [10]:
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _____.
[]
Guessing letter: a
Incorrect guess a, # of tries remaining: 6. Word: _____.
[]
Guessing letter: e
Incorrect guess e, # of tries remaining: 5. Word: _____.
[]
Guessing letter: s
Incorrect guess s, # of tries remaining: 4. Word: _____.
[]
Guessing letter: o
Got a Letter, o, # of tries remaining: 3. Word: ____o.
[({'t'}, 3, 355), ({'r'}, 3, 334), ({'n'}, 3, 289), ({'d'}, 3, 265), ({'l'}, 3, 253), ({'i'}, 3, 249), ({'c'}, 3, 227), ({'g'}, 3, 181), ({'m'}, 3, 109), ({'h'}, 3, 107), ({'b'}, 3, 82), ({'k'}, 3, 70), ({'p'}, 3, 49), ({'z'}, 3, 47), ({'v'}, 3, 47), ({'y'}, 3, 29), ({'f'}, 3, 23), ({'j'}, 3, 21), ({'w'}, 3, 11), ({'x'}, 3, 9), ({'u'}, 3, 8)]
Guessing letter: t
Got a Letter, t, # of tries remaining: 3. Word: _t__o.
[({'p'}, 3, 128), ({'i'}, 3, 119), ({'u'}, 3, 92), ({'c'}, 3, 31), ({'m'}, 3, 16), ({'y'}, 3, 13), ({'l'}, 3, 10), ({'r'}, 3, 10), ({'d'}, 3, 9), ({'f'}, 3, 9), ({'n'}, 3, 9), ({'b'}, 3, 8), ({'g'},

It looks like it is working but man does it get it wrong a lot...

In [ ]:
game.get_winstate()

True

In [5]:
game = MyHangman()

In [6]:
wincount = 0
N = 100
for i in range(N):
    if i % (N/10) == 0:
        print("on game %i of %i" % (i+1, N))
        win_percent = 100*wincount/(i+1)
        print("win percentage: %f" % win_percent)
    game.start_game(verbose=False)
    wincount += game.get_winstate()
print()
print("win percentage: ", 100*wincount/N, "%")

on game 1 of 100
win percentage: 0.000000
on game 11 of 100
win percentage: 63.636364
on game 21 of 100
win percentage: 71.428571
on game 31 of 100
win percentage: 67.741935
on game 41 of 100
win percentage: 68.292683
on game 51 of 100
win percentage: 64.705882
on game 61 of 100
win percentage: 63.934426
on game 71 of 100
win percentage: 61.971831
on game 81 of 100
win percentage: 60.493827
on game 91 of 100
win percentage: 60.439560

win percentage:  61.0 %


In [27]:
print("win percentage: ", 100*wincount/N, "%")

win percentage:  64.0 %


This is a significant improvement... hopefully... but it still isn't good enough in my opinion... I at least have parameters to tweak? It would be easier to mass tweak parameters if it was faster.

Am I doing things wrong? I am still using the FULL dictionary to train the algorithm and test it. I need to test the algorithm on UNKNOWN words. I need to do a test/train split or load another data file.

In [28]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()

In [29]:
tf = open("words_test.txt","r")
test_dict = tf.read().splitlines()
tf.close()

In [30]:
len(full_dict)

227300

In [31]:
len(test_dict)

170671

In [32]:
len(full_dict)/len(test_dict)

1.3318021222117407